In [ ]:
import numpy as np
import pandas as pd
import random
import warnings
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import getpass
from yellowbrick.cluster import KElbowVisualizer
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import time
from spotipy.exceptions import SpotifyException
from pprint import pprint
import gnod_functions as gf



warnings.filterwarnings('ignore')

In [ ]:
pd.set_option("display.max_rows" , 700)
pd.set_option("display.max_columns" , 50)

In [ ]:
import os

spotify_client_id = os.getenv('SPOTIFY CLIENT ID')

# Check if the key is retrieved successfully
if spotify_client_id:
    print("Secret key retrieved successfully!")
else:
    print("Failed to retrieve secret key.")


In [ ]:
spotify_client_secret = os.getenv('SPOTIFY CLIENT SECRET')

# Check if the key is retrieved successfully
if spotify_client_secret:
    print("Secret key retrieved successfully!")
else:
    print("Failed to retrieve secret key.")


In [ ]:
CLIENT_ID = spotify_client_id
CLIENT_SECRET = spotify_client_secret

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))


In [ ]:
"""playlist_id = "37i9dQZF1DWWOaP4H0w5b0"
metal_df = gf.get_features_from_playlist(sp, playlist_id, filename="metal_features.csv")"""

In [ ]:
metal_df = pd.read_csv("metal_features.csv")

In [ ]:
"""related_artists_df = gf.find_related_artists_for_df(sp, metal_df)"""

In [ ]:
"""related_artists_df.to_csv("related_artists.csv", index=False)"""

In [ ]:
related_artists_df = pd.read_csv("related_artists.csv")

In [ ]:
combined_artists = related_artists_df.copy()
combined_artists.rename(columns={"related_artist_name":"artist_name","related_artist_id":"artist_id"}, inplace=True)
combined_df_artists = combined_artists[["artist_id","artist_name"]]
metal_df_artists = metal_df[["artist_id","artist_name"]]
all_artists = pd.concat([combined_df_artists,metal_df_artists])

In [ ]:
all_artists.duplicated().sum()

In [ ]:
unique_artists = all_artists.drop_duplicates(ignore_index=True)
unique_artists_df = unique_artists

In [ ]:
"""top_tracks_df = gf.get_top_tracks_for_artists_in_batches(sp, unique_artists_df)"""

In [ ]:
"""top_tracks_df.to_csv("metal_top_tracks.csv")"""

In [ ]:
"""enhanced_tracks_df = gf.get_audio_features_for_tracks(sp, top_tracks_df)"""

In [ ]:
"""enhanced_tracks_df.to_csv("enhanced_tracks.csv")"""

In [ ]:
enhanced_tracks_df = pd.read_csv("enhanced_tracks.csv")
enhanced_tracks_df.shape

In [ ]:
enhanced_tracks_df.shape

In [ ]:
enhanced_tracks_df.duplicated().sum()

In [ ]:
duplicates = enhanced_tracks_df[enhanced_tracks_df.duplicated(keep=False)]
duplicates

In [ ]:
enhanced_tracks_df.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
enhanced_tracks_df.shape

## SOME EDA

In [ ]:
sns.histplot(enhanced_tracks_df['tempo'], kde=False, color='blue')

plt.xlabel("Tempo (BPM)", fontsize=14)

plt.ylabel("Count", fontsize=14)

plt.grid(False)

# Save the plot
plt.savefig("tempo_update.png", dpi=300)

plt.show()

In [ ]:
sns.histplot(enhanced_tracks_df['valence'], kde=False, color='blue')

plt.xlabel("Valence", fontsize=14)

plt.ylabel("Count", fontsize=14)

plt.grid(False)

# Save the plot
plt.savefig("valence_update.png", dpi=300)

plt.show()

In [ ]:
sns.histplot(enhanced_tracks_df['energy'], kde=False, color='blue')

plt.xlabel("Energy", fontsize=14)

plt.ylabel("Count", fontsize=14)

plt.grid(False)

# Save the plot
plt.savefig("energy_update.png", dpi=300)

plt.show()

In [ ]:
sns.histplot(enhanced_tracks_df['tempo'], kde=False, color='blue')

plt.xlabel("Tempo (BPM)", fontsize=14)

plt.ylabel("Count", fontsize=14)

plt.grid(False)

# Save the plot
plt.savefig("tempo_update.png", dpi=300)

plt.show()

In [ ]:
sns.histplot(enhanced_tracks_df['loudness'], kde=False, color='blue')

plt.xlabel("Loudness (dB)", fontsize=14)

plt.ylabel("Count", fontsize=14)

plt.grid(False)

# Save the plot
plt.savefig("loudness_update.png", dpi=300)

plt.show()

In [ ]:
sns.histplot(enhanced_tracks_df['danceability'], kde=False, color='blue')

plt.xlabel("Danceability", fontsize=14)

plt.ylabel("Count", fontsize=14)

plt.grid(False)

# Save the plot
plt.savefig("danceability_update.png", dpi=300)

plt.show()

In [ ]:
sns.histplot(enhanced_tracks_df['instrumentalness'], kde=False, color='blue')

plt.xlabel("Instrumentalness", fontsize=14)

plt.ylabel("Count", fontsize=14)

plt.grid(False)

# Save the plot
plt.savefig("instrumentalness_update.png", dpi=300)

plt.show()

In [ ]:
cluster_df = enhanced_tracks_df.select_dtypes(include=np.number)
cluster_df.head()

In [ ]:
cluster_df_slim = cluster_df.drop(columns = ["track_popularity","track_number","key","mode","duration_ms","time_signature"])
cluster_df_slim

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cluster_df_slim_scaled = scaler.fit_transform(cluster_df_slim)
cluster_df_slim_scaled = pd.DataFrame(cluster_df_slim_scaled, columns=cluster_df_slim.columns)

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn import cluster, datasets

In [ ]:
model = cluster.KMeans(random_state=42, init="random")
visualizer = KElbowVisualizer(model, k=(2,20))
visualizer.fit(cluster_df_slim_scaled)
visualizer.poof()

In [ ]:
#Optimal amount of clusters = 7 !

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=7, random_state=42)

cluster_labels = kmeans.fit_predict(cluster_df_slim_scaled)

In [ ]:
cluster_df_slim['Cluster'] = cluster_labels

In [ ]:
cluster_df_slim

In [ ]:
print(cluster_df_slim['Cluster'].value_counts())

In [ ]:
from sklearn.metrics import silhouette_score

# Calculate the silhouette score for your clustering
score = silhouette_score(cluster_df_slim_scaled, cluster_labels)
print("Silhouette Score:", score)


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduce dimensions to 2D for visualization
pca = PCA(n_components=2)
pca_components = pca.fit_transform(cluster_df_slim_scaled)

# Plot the clusters in 2D
plt.figure(figsize=(10, 7))
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=cluster_labels, cmap='viridis', s=50)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('KMeans Clusters (PCA Reduced)')
plt.colorbar(label='Cluster Label')
plt.show()

In [ ]:
# Matplotlib
plt.matshow(pca.components_, cmap='viridis')
plt.yticks([0, 1], ["First component", "Second component"])
plt.colorbar()
plt.xticks(range(len(cluster_df_slim_scaled.columns)), cluster_df_slim_scaled.columns, rotation=90)
plt.show()

In [ ]:
from sklearn.manifold import TSNE

# Reduce dimensions to 2D for visualization with t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_components = tsne.fit_transform(cluster_df_slim_scaled)

# Plot the clusters in 2D
plt.figure(figsize=(10, 7))
plt.scatter(tsne_components[:, 0], tsne_components[:, 1], c=cluster_labels, cmap='viridis', s=50)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('KMeans Clusters (t-SNE Reduced)')
plt.colorbar(label='Cluster Label')
plt.show()

In [ ]:
# Get cluster centers in original scaled dimensions
cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=cluster_df_slim.columns[:-1])
print(cluster_centers)

In [ ]:
import pandas as pd

# Add cluster labels to the DataFrame
cluster_df_slim_scaled['Cluster'] = cluster_labels

# Calculate within-cluster variance for each feature
feature_influence = cluster_df_slim_scaled.groupby('Cluster').var().mean()
print("Feature Influence based on Within-Cluster Variance:\n", feature_influence)

In [ ]:
enhanced_tracks_df["Cluster"] = cluster_labels

In [ ]:
enhanced_tracks_df.to_csv("enhanced_tracks_cluster7.csv")

In [ ]:
simplified_cluster_df = enhanced_tracks_df[["artist_name","track_name","Cluster","energy","danceability","tempo"]]
simplified_cluster_df.head(100)

In [ ]:
#reducing the amount of features to the most relevant

### REDUCING THE AMOUNT OF FEATURES TO THE MOST RELEVANT, INCREASING CLUSTER AMOUNT

In [ ]:
cluster_df_slim = cluster_df.drop(columns = ["track_popularity","track_number","key","mode","duration_ms","time_signature"])

In [ ]:
cluster_df_reduced = cluster_df_slim.drop(columns=["instrumentalness","liveness","acousticness","speechiness"])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cluster_df_reduced_scaled = scaler.fit_transform(cluster_df_reduced)
cluster_df_reduced_scaled = pd.DataFrame(cluster_df_reduced_scaled, columns=cluster_df_reduced.columns)

In [ ]:
cluster_df_reduced_scaled

In [ ]:
cluster_df_further_reduced_scaled = cluster_df_reduced_scaled.drop(columns="danceability")

In [ ]:
cluster_df_further_reduced_scaled

In [ ]:
from sklearn import cluster
from yellowbrick.cluster import KElbowVisualizer
import matplotlib.pyplot as plt

# Initialize the KMeans model
model = cluster.KMeans(random_state=42, init="random")

# Create the elbow visualizer
visualizer = KElbowVisualizer(model, k=(2, 20))

# Fit the data to the visualizer
visualizer.fit(cluster_df_further_reduced_scaled)

# Save the figure
visualizer.finalize()  # Ensure the visualizer has fully rendered its labels and title
plt.savefig("elbow_visualizer.png", dpi=300, bbox_inches='tight')  # Save with all elements

# Show the plot (optional)
plt.show()

# Close the plot to clean up resources (if running multiple plots in sequence)
plt.close()



In [ ]:
#Recommended number of clusters 6 by Distortion Score Elbow, but testing and evaluating with domain knowledge 10 clusters yielded the
#best results for useful separation and real-world meaningfulness of the clusters

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=42)

cluster_labels = kmeans.fit_predict(cluster_df_further_reduced_scaled)

In [ ]:
cluster_df_slim['Cluster'] = cluster_labels

In [ ]:
from sklearn.metrics import silhouette_score

# Calculate the silhouette score for your clustering
score = silhouette_score(cluster_df_further_reduced_scaled, cluster_labels)
print("Silhouette Score:", score)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduce dimensions to 2D for visualization
pca = PCA(n_components=2)
pca_components = pca.fit_transform(cluster_df_further_reduced_scaled)

# Plot the clusters in 2D
plt.figure(figsize=(10, 7))
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=cluster_labels, cmap='viridis', s=50)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('KMeans Clusters (PCA Reduced)')
plt.colorbar(label='Cluster Label')
plt.savefig("PCA_further_reduced10.png", dpi=300)
plt.show()


In [ ]:
# Matplotlib
plt.matshow(pca.components_, cmap='viridis')
plt.yticks([0, 1], ["First component", "Second component"])
plt.colorbar()
plt.xticks(range(len(cluster_df_further_reduced_scaled.columns)), cluster_df_further_reduced_scaled.columns, rotation=90)
plt.show()

# Plotly
# fig = px.imshow(pca.components_, y=['1st component', '2nd component', '3rd component'], x=list(data.columns.drop('label')), color_continuous_scale='viridis')
# fig.update_coloraxes(overwrite=True)
# fig.update_layout(showlegend=False)
# fig.show()

In [ ]:
from sklearn.manifold import TSNE

# Reduce dimensions to 2D for visualization with t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_components = tsne.fit_transform(cluster_df_further_reduced_scaled)

# Plot the clusters in 2D
plt.figure(figsize=(10, 7))
plt.scatter(tsne_components[:, 0], tsne_components[:, 1], c=cluster_labels, cmap='viridis', s=50)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('KMeans Clusters (t-SNE Reduced)')
plt.colorbar(label='Cluster Label')
plt.savefig("tSNE_further_reduced_10.png")
plt.show()

In [ ]:
print(cluster_df_slim['Cluster'].value_counts())

In [ ]:
import pandas as pd

# Add cluster labels to the DataFrame
cluster_df_further_reduced_scaled['Cluster'] = cluster_labels

# Calculate within-cluster variance for each feature
feature_influence = cluster_df_further_reduced_scaled.groupby('Cluster').var().mean()
print("Feature Influence based on Within-Cluster Variance:\n", feature_influence)


In [ ]:
enhanced_tracks_df["Cluster"] = cluster_labels

In [ ]:
enhanced_tracks_df.to_csv("enhanced_tracks_cluster10_further_reduced.csv")

In [ ]:
simplified_cluster_df = enhanced_tracks_df[["artist_name","track_name","Cluster","energy","danceability","tempo"]]
simplified_cluster_df.head(200)